In [1]:
import pandas as pd
import datetime 

Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.




In [2]:
df = pd.read_csv('ml-latest-small/ratings.csv')
df

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [3]:
#Находим LifeTime для всех userId и агрегируем min и max значений timestamp, и находим [ltv]
# отбрасываем лишние столбцы
user_ts = df.groupby('userId').agg(['min', 'max'])['timestamp']
user_ts['life_time'] = user_ts['max']-user_ts['min'] 

user_ts

,min,max,life_time
userId,,,
1,1260759108,1260759205,97
2,835355395,835356246,851
3,1298861589,1298932787,71198
4,949778714,949982274,203560
5,1163373044,1163375145,2101
...,...,...,...
667,847271220,847272234,1014
668,993613196,993613478,282
669,1015829081,1015829766,685


In [4]:
#Групируем значения по userId с подсчётом количества голосов, отбрасываем лишние столбцы,
#сортируем по количеству голосов, переименовываем для удобства
top_voters = df.groupby('userId').count().sort_values('movieId', ascending= False)

top_voters.rename(columns={'movieId': 'votes'}, inplace=True)

top_voters = top_voters[top_voters['votes']>100]

top_voters

,votes,rating,timestamp
userId,,,
547,2391,2391,2391
564,1868,1868,1868
624,1735,1735,1735
15,1700,1700,1700
73,1610,1610,1610
...,...,...,...
217,104,104,104
36,104,104,104
623,103,103,103


In [5]:
#Объединяю таблицы через .join так как (userID)индексы, хотя можно и через .merge по userId
lt = top_voters.join(user_ts, how='left').drop(['rating','timestamp'],axis=1).reset_index().drop(['min','max'],axis=1)

print('Среднее время жизни пользователей:', datetime.timedelta(seconds= lt['life_time'].mean()))

lt


Среднее время жизни пользователей: 463 days, 21:28:27.449612


,userId,votes,life_time
0,547,2391,501810535
1,564,1868,1516358
2,624,1735,457492507
3,15,1700,471393496
4,73,1610,220585493
...,...,...,...
253,217,104,981
254,36,104,5949290
255,623,103,3102
256,257,103,111934874


In [6]:
def life_counter(life_time):
    return datetime.timedelta(seconds= int(life_time['life_time']))

lt['time'] = lt.apply(life_counter,axis=1)
lt.drop(['life_time'],axis=1,inplace=True)

lt    

,userId,votes,time
0,547,2391,5807 days 23:48:55
1,564,1868,17 days 13:12:38
2,624,1735,5295 days 01:15:07
3,15,1700,5455 days 22:38:16
4,73,1610,2553 days 01:44:53
...,...,...,...
253,217,104,0 days 00:16:21
254,36,104,68 days 20:34:50
255,623,103,0 days 00:51:42
256,257,103,1295 days 13:01:14


Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
1.таблицу с тремя типами выручки для каждого client_id без указания адреса клиента:
2. аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:
rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

In [7]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [8]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [9]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [10]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [11]:
joined_tab_1 = rzd.merge(auto, how='outer', on='client_id').merge(air, how='outer', on='client_id')

print('RZD ',joined_tab_1.rzd_revenue.sum()== rzd.rzd_revenue.sum())#Control
print('Auto ',joined_tab_1.auto_revenue.sum()== auto.auto_revenue.sum())#Control
print('Air ',joined_tab_1.air_revenue.sum()== air.air_revenue.sum())#Control

joined_tab_1


RZD  True
Auto  True
Air  True


,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [12]:
joined_tab_2 = joined_tab_1.merge(client_base, how='outer', on= 'client_id')
joined_tab_2

#Хорошо бы NaN-ы заменить на что нибуть, хотя смотря как дальше использоватьсябудет таблица, если для расчётов то лучше с ними 

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


# Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Не уверен что правильно понял задание, но вот мои размышления:
1. "...для каждого user_id известна последовательность координат (широта/долгота)...",  мы знаем только координаты или навигационные данные? обычно данные с навигаторов содержат не только координаты но и как минимум, время UTC, по которому можно отсортировать координаты и получить временной трек, Регистрация визита подразумевает наличие, как минимум, информацию user_id, а регистрация факта покупки, как минимум содержит время операции, по этому при объединении таблиц, логично выстроить следующий порядок действий:
                1. Выделяем из строки с навигационными данными для каждого user_id(с помощью RegEx, или .split)  данные [Время], [Широта], [Долгота]. Сортировку(индексирование) делаем по времени. Получаем "трек" перемещения конкретного user_id.
                2.1 Если таблица визитов содержит данные времени визита: Объединяем(через .merge) таблицу визитов( [Время визита][user_iD]) с таблицей навигационных данных по столбцу [Время][Время визита], с параметром 'Inner', получаем таблицу структуры:
                [Время][user_iD][Широта], [Долгота] то есть навигационные данные на момент визита конкретного user_id
                2.2 Если таблица визитов не содержит времени визита: можно попробовать через функцию в таблице с навигационными данными найти самые близкие к месту визита координаты и вернуть время визита с широтой и долготой и добавить эти данные в таблицу визитов: Таким образом таблица визитов будет иметь вид [Время][user_iD][Широта], [Долгота]
                
                3.Предполагается что таблица покупок должна содержать данные о времени покупки и сумме покупки:
                Если имеется идентификация покупателя по user_iD, то объединение делаем по user_id, если таблица покупок не содержит данных о покупателе а содержит только время и сумму, то опять же через функцию пытаемся по времени покупки найти  таблице визитов найти наиболее близкие значения визитов и добавляем столбцы покупки в общую таблицу, получаем некий датафрейм: [Время][user_iD][Широта][Долгота][Сумма покупки].
                
                
                Надеюсь что правильно понял задание





2.Если данные таблиц не содержат отметок времени, то можно попробовать объединить таблицу координат и таблицу визитов по полю к  

In [15]:
geo = pd.DataFrame(
    {
        'user_id': [1, 2, 3, 4, 5],
        'geo_data': ['10/11', '20/12', '30/13','40/14','50/15']
    }
)
geo

,user_id,geo_data
0,1,10/11
1,2,20/12
2,3,30/13
3,4,40/14
4,5,50/15
